In [ ]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv("fire_nrt_J1V-C2_684138.csv")

# Ensure columns exist – adjust names if needed
# Typical FIRMS columns: latitude, longitude, acq_date, acq_time, instrument, etc.
df['acq_datetime'] = pd.to_datetime(
    df['acq_date'] + ' ' + df['acq_time'].astype(str).str.zfill(4),
    format='%Y-%m-%d %H%M'
)





# Round coordinates to create "cells" of ~0.05 degrees (~5 km)
df['lat_cell'] = (df['latitude'] / 0.05).round().astype(int)
df['lon_cell'] = (df['longitude'] / 0.05).round().astype(int)

# Group fires by day and spatial cell
df['date'] = df['acq_datetime'].dt.date
groups = df.groupby(['date', 'lat_cell', 'lon_cell'])


events = []

for (date, lat_cell, lon_cell), g in groups:
    lat_min = g['latitude'].min()
    lat_max = g['latitude'].max()
    lon_min = g['longitude'].min()
    lon_max = g['longitude'].max()

    # Add a small buffer (~0.02 deg ≈ ~2 km)
    buffer = 0.02
    lat_min -= buffer
    lat_max += buffer
    lon_min -= buffer
    lon_max += buffer

    # Time window: same day ± 1 day
    t_start = datetime.combine(date, datetime.min.time()) - timedelta(days=1)
    t_end   = datetime.combine(date, datetime.max.time()) + timedelta(days=1)

    events.append({
        "bbox": [lon_min, lat_min, lon_max, lat_max],
        "time_interval": (t_start.strftime("%Y-%m-%d"), t_end.strftime("%Y-%m-%d")),
        "n_firms_points": len(g)
    })

len(events)






In [ ]:
from sentinelhub import SHConfig, SentinelHubCatalog, BBox, CRS, DataCollection

config = SHConfig()

config.sh_client_id = "d582ede6-aa36-4e58-b607-162d333d9d19"
config.sh_client_secret = "eiojycDQr6FzktTpqLgO7N3uFQckVr7R"

catalog= SentinelHubCatalog(config=config) 


s2_events = []
rate_limit = 10
for ev in events:
    if rate_limit <= 0:
        break
    rate_limit -= 1
    bbox = BBox(ev["bbox"], crs=CRS.WGS84)
    time_interval = ev["time_interval"]   # ('YYYY-MM-DD', 'YYYY-MM-DD')

    search_iterator = catalog.search(
        DataCollection.SENTINEL2_L2A,
        bbox=bbox,
        time=time_interval,
        # Option A: no server-side cc filter, filter later
        fields={
            "include": ["id", "properties.datetime", "properties.eo:cloud_cover"],
            "exclude": []
        },
        limit=10
    )

    items = list(search_iterator)
    # Optional client-side cc filter:
    items = [
        it for it in items
        if it["properties"].get("eo:cloud_cover", 100) < 70
    ]

    if not items:
        continue

    s2_events.append({
        "firms_event": ev,
        "s2_items": items
    })


for e in s2_events[:5]:
    print("FIRMS event:", e['firms_event']['time_interval'], e['firms_event']['bbox'])
    for it in e['s2_items']:
        print("  S2:", it['id'], it['properties']['datetime'],
              "cloud:", it['properties']['eo:cloud_cover'])
    print()
# Create bounding boxes with buffer and time intervals

FIRMS event: ('2025-11-02', '2025-11-04') [np.float64(-67.94972), np.float64(-46.807660000000006), np.float64(-67.90972000000001), np.float64(-46.76766)]
  S2: S2C_MSIL2A_20251104T141101_N0511_R110_T19GEJ_20251104T171509 2025-11-04T14:25:58Z cloud: 6.6



In [ ]:
from sentinelhub import SentinelHubRequest, DataCollection, MimeType, bbox_to_dimensions

evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["B04", "B08", "B12", "SCL"],
    output: { bands: 4, sampleType: "FLOAT32" }
  };
}
function evaluatePixel(sample) {
  return [sample.B04, sample.B08, sample.B12, sample.SCL];
}
"""

def download_s2_for_event(event_idx, ev, item, out_dir):
    bbox = BBox(ev["bbox"], crs=CRS.WGS84)
    resolution = 20  # m
    size = bbox_to_dimensions(bbox, resolution=resolution)

    acq_date = item["properties"]["datetime"][:10]

    request = SentinelHubRequest(
        evalscript=evalscript,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A,
                time_interval=(acq_date, acq_date),
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=bbox,
        size=size,
        config=config,
        data_folder=out_dir,   # ✅ save location
    )

    # Option A: get array AND save to disk
    arr = request.get_data(save_data=True)[0]

    # Option B: if you only care about files on disk
    # request.save_data()

    # Inspect where it ended up
    filenames = request.get_filename_list()
    print(f"Saved files for event {event_idx}:")
    for f in filenames:
        print("  ", f)

    return arr, filenames

# Example: download first scene per event
for idx, e in enumerate(s2_events):
    ev = e["firms_event"]
    item = e["s2_items"][0]  # or pick the best (lowest cloud etc.)
    out_path = f"s2_fire_event_{idx}.tif"
    download_s2_for_event(idx, ev, item, out_path)


TypeError: DataRequest.save_data() takes 1 positional argument but 2 were given